# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Loading Libraries

You should restart the runtime after running the cell below.

In [2]:
!pip install progressbar2
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [3]:
# Checking if the keras version is OK
import keras

if keras.__version__  == '2.2.4':
  print('Version ok!')
else:
  print('Should be v2.2.4')

Using TensorFlow backend.


Version ok!


In [0]:
import progressbar
from scipy import misc
import numpy as np
import os
import math
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import MobileNet
from keras import Model
from keras.layers import Conv2D, BatchNormalization, ReLU, GlobalAveragePooling2D

# Loading the model

The `imagenet` weights are not available right now, so the model its been loaded with the random weights from a normal distribution.

## MobileNet V2

In [0]:
modelV2 = MobileNetV2(input_shape=(160, 160, 3),
           alpha=1,
           depth_multiplier=1,
           weights=None,
           include_top=False,
           pooling=None)

In [0]:
inp = modelV2.input
x = modelV2.layers[-4].output

x = Conv2D(filters=512, kernel_size=(5,5), strides=1, padding="same", activation=None, 
           name="Conv_Last", use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = GlobalAveragePooling2D()(x)

v2_model = Model(inputs=inp, outputs=x)


## MobileNet V1

Loading the `MobileNetV1` implementation from Keras, with the weights pre-trained on the **ImageNet** classification challenge.

In [5]:
modelV1 = MobileNet(input_shape=(160, 160, 3),
                   alpha=1,
                   depth_multiplier=1,
                   weights='imagenet',
                   dropout=0,
                   include_top=False,
                   pooling=None)

17227776/17225924 [==============================] - 1s 0us/step


In [0]:
inp = modelV1.input
x = modelV1.layers[-1].output
x = Conv2D(filters=512, kernel_size=(5, 5), strides=1, padding="same", activation=None,
          name="Conv_Last", use_bias=False)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = GlobalAveragePooling2D()(x)

v1_model = Model(inputs=inp, outputs=x)

## Summary

Running the cell below it's an easy way to visualize each layer of the Convolution Neural Network, together with its input and output tensor shapes.

In [0]:
modelV1.summary()

In [0]:
v1_model.summary()

In [0]:
modelV2.summary()

In [0]:
final_model.summary()

# Plotting model

Responsible for plotting the loaded model for a better visualization of its parameters.

In [0]:
# import graphviz
# import pydot

!pip install pydot
!pip install graphviz

In [0]:
from keras.utils import plot_model

In [0]:
plot_model(modelV2, to_file='/gdrive/My Drive/FaceRecognition/art/mobilenet_v2.png', show_shapes=True, show_layer_names=True)

# Input Data

Getting a list of all the input images path. Before running this, check if the `lfw` folder already has the `input.npy` and `output.npy` files. If so, go to the **Splitting the Data** section.

In [0]:
initial_path = '/gdrive/My Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'
os.listdir(initial_path)

dirs = [os.path.join(initial_path, d) for d in os.listdir(initial_path) if os.path.isdir(os.path.join(initial_path, d))]

KeyboardInterrupt: ignored

In [0]:
import progressbar

inputs = []
for d in progressbar.progressbar(dirs):
  for f in os.listdir(d):
    if f.endswith('png') or f.endswith('jpg') or f.endswith('jpeg'):
      inputs.append(os.path.join(d, f))

In [0]:
outputs = []

for inp in progressbar.progressbar(inputs):
  filename = inp.split('/')[-1]
  path = inp.split('/')[:-1]
  
  filename = 'output/' + filename.split('.')[0] + '.npy'
  path = '/'.join(path)
  
  outputs.append(os.path.join(path, filename))

In [0]:
inputs_np = np.array(inputs)
outputs_np = np.array(outputs)

np.save('/gdrive/My Drive/FaceRecognition/datasets/lfw/input.npy', inputs_np)
np.save('/gdrive/My Drive/FaceRecognition/datasets/lfw/output.npy', outputs_np)

# Splitting the Data

Splitting the data on training, validation and test sets. It should follow the partition as 80% - 10% - 10%.

In [0]:
input_data = np.load('/gdrive/My Drive/FaceRecognition/datasets/lfw/input_resnet.npy')
output_data = np.load('/gdrive/My Drive/FaceRecognition/datasets/lfw/output_resnet.npy')

In [0]:
nrof_samples = input_data.shape[0]
randomize = np.random.peyrmutation(nrof_samples)
inp = input_data[randomize]
out = output_data[randomize]

In [0]:
training_size = np.floor(.8 * nrof_samples).astype(int)
val_size = (nrof_samples - training_size) // 2

X_train = inp[:training_size]
y_train = out[:training_size]
X_val = inp[training_size:training_size+val_size]
y_val = out[training_size:training_size+val_size]
X_test = inp[training_size+val_size:]
y_test = out[training_size+val_size:]

# Training Callback

A way to visualize the training `loss` in real time. 

In [0]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

class TrainingPlot(keras.callbacks.Callback):
  # This function is called when the training begins
  def on_train_begin(self, logs={}):
    self.losses = []
    self.val_losses = []
    self.logs = []
    
  # This function is called at the end of each epoch
  def on_epoch_end(self, epoch, logs={}):
    # Append thel logs, losses to the lists
    self.logs.append(logs)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))
    
    # Before plotting ensure at least 2 epochs have passed
    if len(self.losses) > 1:
      # Clear the previous plot
      clear_output(wait=True)
      N = np.arange(0, len(self.losses))
      
      plt.style.use('seaborn')
      
      plt.figure()
      plt.plot(N, self.losses, label="train_loss")
      plt.plot(N, self.val_losses, label="val_loss")
      plt.title("Training & Val Loss [Epoch {}]".format(epoch))
      plt.xlabel("Epoch #")
      plt.ylabel("Loss")
      plt.legend()
      plt.show()
      
plot_losses = TrainingPlot()

# Training

Distilling the network of the pre-trained [Facenet](https://github.com/davidsandberg/facenet) into a MobileNet v2.

## Loading the input generator

`TCC_Generator` class has been developed to manage the input images through the CNN during training. It's responsible to split the training set into a series of batches (which its lengths has been previously defined), and then feed those batches (X and y, respectively) through the Mobile Net training architecture.

In [0]:
os.chdir('/gdrive/My Drive/FaceRecognition/src')
from data_generator import TCCGenerator

In [0]:
batch_size = 128
training_batch_generator = TCCGenerator(X_train, y_train, batch_size)
val_batch_generator = TCCGenerator(X_val, y_val, batch_size)

## Compile and Training the model

Compile the model with the Adam optimizer and learning rate $0.001$. The loss function is **Mean Squared Error**, and the training will run for 40 epochs using 10 processes on different threads. 

### MobineNet V2

In [0]:
from keras import losses

num_epochs = 100
queue_size = 12

v2_model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                   loss=losses.mean_squared_error)

In [0]:
history = v2_model.fit_generator(generator=training_batch_generator,
                         epochs=num_epochs,
                         verbose=1,
                         validation_data=val_batch_generator,
                         max_queue_size=queue_size,
                         workers=10)

Epoch 1/100
83/83 [==============================] - 1890s 23s/step - loss: 0.0230 - val_loss: 0.0045
Epoch 2/100
83/83 [==============================] - 85s 1s/step - loss: 0.0035 - val_loss: 0.0041
Epoch 3/100
83/83 [==============================] - 88s 1s/step - loss: 0.0020 - val_loss: 0.0025
Epoch 4/100
83/83 [==============================] - 84s 1s/step - loss: 0.0019 - val_loss: 0.0020
Epoch 5/100
83/83 [==============================] - 83s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 6/100
83/83 [==============================] - 86s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 7/100
83/83 [==============================] - 86s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 8/100
83/83 [==============================] - 87s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 9/100
83/83 [==============================] - 84s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 10/100
83/83 [==============================] - 84s 1s/step - loss: 0.0019 - val_loss: 0.0019
Epoch 

In [0]:
# Saving the trained model

v2_model.save('/gdrive/My Drive/FaceRecognition/models/mobile-net/mobilenetv2_v1.h5')

### MobileNet V1

In [0]:
learning_rates = [0.05, 0.03, 0.01, 0.005, 0.001]

batch_size = 128
num_epochs = 5
queue_size = 12

Wsave = v1_model.get_weights()
for idx, learning_rate in enumerate(learning_rates):
  print('[ITERATION {}] Learning rate: {}'.format(idx+1, learning_rate))
  v1_model.compile(optimizer=keras.optimizers.Nadam(lr=learning_rate),
                  loss=losses.mean_squared_error,
                   metrics=[losses.mean_absolute_error])
  
  v1_model.fit_generator(generator=training_batch_generator,
                       steps_per_epoch=(training_size // batch_size),
                       epochs=num_epochs,
                       verbose=1,
                       validation_data=val_batch_generator,
                       validation_steps=(val_size // batch_size),
                       max_queue_size=queue_size,
                       workers=10)
  print('Setting the weights back... \n\n')
  v1_model.set_weights(Wsave)
  
  

In [0]:
from keras import losses

batch_size = 128
num_epochs = 70
queue_size = 12

v1_model.compile(optimizer=keras.optimizers.Nadam(lr=0.001),
                   loss=losses.mean_squared_error,
                   metrics=[losses.cosine_proximity,
                           losses.mean_absolute_error])

In [0]:
v1_model.fit_generator(generator=training_batch_generator,
                       epochs=num_epochs,
                       verbose=1,
                       validation_data=val_batch_generator,
                       validation_steps=(val_size // batch_size),
                       max_queue_size=queue_size,
                       workers=10)

Epoch 1/70
83/83 [==============================] - 99s 1s/step - loss: 0.0259 - cosine_proximity: -0.0141 - mean_absolute_error: 0.1065 - val_loss: 0.0039 - val_cosine_proximity: 0.0236 - val_mean_absolute_error: 0.0498
Epoch 2/70
83/83 [==============================] - 90s 1s/step - loss: 0.0020 - cosine_proximity: -0.1228 - mean_absolute_error: 0.0361 - val_loss: 0.0027 - val_cosine_proximity: 1.8022e-05 - val_mean_absolute_error: 0.0406
Epoch 3/70
83/83 [==============================] - 83s 1s/step - loss: 0.0020 - cosine_proximity: -0.1354 - mean_absolute_error: 0.0360 - val_loss: 0.0020 - val_cosine_proximity: -0.0474 - val_mean_absolute_error: 0.0360
Epoch 4/70
83/83 [==============================] - 82s 986ms/step - loss: 0.0019 - cosine_proximity: -0.1538 - mean_absolute_error: 0.0350 - val_loss: 0.0019 - val_cosine_proximity: -0.1632 - val_mean_absolute_error: 0.0350
Epoch 5/70
83/83 [==============================] - 83s 1000ms/step - loss: 0.0019 - cosine_proximity: -0.1

In [0]:
# Saving the trained model

v1_model.save('/gdrive/My Drive/FaceRecognition/models/mobile-net/mobilenetv1_v1.h5')

# Validating

We have 1324 images on the test set. First we need to identify each individual and then run forward propagation on our model on all of our images. And then, using our test set, we will check, based on our known distance function (calculates an euclidean distance or cosine similarity between two embeddings), which predictions our model is capable of doing. 

In [0]:
# Saving the X_test used for the model_v1 and model_v2 h5 files.
np.save('/gdrive/My Drive/FaceRecognition/datasets/lfw/xtest.npy', X_test)
np.save('/gdrive/My Drive/FaceRecognition/datasets/lfw/ytest.npy', y_test)

## Running forward propagation

Running forward propagation on all images. The images will be saved as `output_v/` where *v* is the version of its model. The desired shape on each embedding is `(1, 512)`

## Mobilenetv2 V1

Running on MobilenetV2 trained with learning rate 0.001 and for 100 epochs.

In [0]:
model = keras.models.load_model('/gdrive/My Drive/FaceRecognition/models/mobile-net/mobilenetv2_v1.h5')

In [0]:
base_path = '/gdrive/My Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/'
list_folders = os.listdir(base_path)
list_folders = [os.path.join(base_path, x) for x in list_folders]

for folder in progressbar.progressbar(list_folders):
  if not os.path.isdir(folder):
    continue
    
  list_images = os.listdir(folder)
  list_images = [os.path.join(folder, image) for image in list_images]
  list_images = list(filter(lambda x: os.path.isfile(x), list_images))
  filenames = [x.split('/')[-1].split('.')[0] for x in list_images]
  output_filenames = [x + '.npy' for x in filenames]
  output_folder = os.path.join(folder, 'mobilenetv2_v1')
  
  # Get the embeddings
  images = np.array([misc.imread(f) for f in list_images])
  embeddings = model.predict(images)
  
  if not os.path.exists(os.path.join(base_path, output_folder)):
    os.makedirs(os.path.join(base_path, output_folder))
  for idx, embedding in enumerate(embeddings):
    emb_to_save = embedding.reshape(1, *embedding.shape)
    np.save(os.path.join(output_folder, output_filenames[idx]), emb_to_save)

100% (5754 of 5754) |####################| Elapsed Time: 0:20:42 Time:  0:20:42


## Mobilenetv1 V1

Running on MobilenetV1 trained with learning rate 0.001 and for 70 epochs

In [0]:
model = keras.models.load_model('/gdrive/My Drive/FaceRecognition/models/mobile-net/mobilenetv1_v1.h5')

In [0]:
base_path = '/gdrive/My Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/'
list_folders = os.listdir(base_path)
list_folders = [os.path.join(base_path, x) for x in list_folders]

for folder in progressbar.progressbar(list_folders):
  if not os.path.isdir(folder):
    continue
    
  list_images = os.listdir(folder)
  list_images = [os.path.join(folder, image) for image in list_images]
  list_images = list(filter(lambda x: os.path.isfile(x), list_images))
  filenames = [x.split('/')[-1].split('.')[0] for x in list_images]
  output_filenames = [x + '.npy' for x in filenames]
  output_folder = os.path.join(folder, 'mobilenetv1_v1')
  
  # Get the embeddings
  images = np.array([misc.imread(f) for f in list_images])
  embeddings = model.predict(images)
  
  if not os.path.exists(os.path.join(base_path, output_folder)):
    os.makedirs(os.path.join(base_path, output_folder))
  for idx, embedding in enumerate(embeddings):
    emb_to_save = embedding.reshape(1, *embedding.shape)
    np.save(os.path.join(output_folder, output_filenames[idx]), emb_to_save)

  0% (9 of 5754) |                       | Elapsed Time: 0:00:00 ETA:   0:09:20

In [0]:
model

## Running Validation

In [0]:
# Loading the dataset
dataset = np.load('/gdrive/My Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/embeddings_test_mac.npy')

In [0]:
def distance(embeddings1, embeddings2, distance_metric='euclidean'):
    """ Calculate the distance between two embeddings. Currently working with euclidean and cosine similarity. 

        :param embeddings1: First embedding
        :param embeddings2: Second embedding
        :param distance_metric: Distance metric to be used to make the calculation. Should be either: 'euclidean' or 'cosine'

        :returns: The distance between the `embeddings1` and `embeddings2`
    """
    assert distance_metric in ['euclidean', 'cosine'], "The distance metric should be either 'euclidean' or 'cosine'"

    if distance_metric == 'euclidean':
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff), 1)

    elif distance_metric == 'cosine':
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=1)
        norm = np.linalg.norm(embeddings1, axis=1) * np.linalg.norm(embeddings2, axis=1)
        similarity = dot / norm
        dist = np.arccos(similarity) / math.pi

    return dist

In [0]:
def only_alpha(string):
  return all(not a.isdigit() for a in string)

def clean_name(path_name, with_number=False):
  path_name = path_name.split('/')[-1]
  path_name = path_name.split('.')[0]
  
  if with_number:
    return path_name
  
  list_names = path_name.split('_')
  
  list_names = list(filter(lambda x: only_alpha(x), list_names))
  name = '_'.join(list_names)
  return name

In [0]:
def get_names(data):
  """ Return the list of unique names that compose the dataset
  
      :params data: The dataset to be analyzed
  """
  names = []
  for image_dict in data:
    name = clean_name(image_dict['name'])
    
    if name not in names:
      names.append(name)
      
  idxs = np.arange(len(names))
  name_to_idx = dict(zip(iter(names), iter(idxs)))
  return np.array(names), name_to_idx

In [0]:
def is_diff(x, y):
  return x.split('/')[-1].split('.')[0] != y.split('/')[-1].split('.')[0]

In [0]:
def predict_face(image_path,
                embedding,
                dataset,
                threshold=.1,
                distance_metric='cosine'):
  people, name_to_idx = get_names(dataset)
  y_hat = clean_name(image_path)
  distances = np.zeros(len(people))
  heap_modified = np.zeros(len(people))
  
  for image_dict in dataset:
    if is_diff(image_dict['name'], image_path):
      continue
      
    name = clean_name(image_dict['name'])
    d = distance(embedding, image_dict['embedding'], distance_metric=distance_metric)
    idx = name_to_idx[name]
    
    if heap_modified[idx] == 0:
      heap_modified[idx] = 1
      distances[idx] = d
    else:
      if distances[idx] > d:
        distances[idx] = d
        
#   idx_min = distances.argmin()
  idxs = np.argsort(distances)[:5]
  y = [people[idx] for idx in idxs]  
  match = y_hat in y
#   y = people[idx_min]
#   match = y_hat == y
  return match, y, y_hat

In [0]:
x = np.array([15, 12, 10, 2])
np.argsort(x)

In [0]:
# def predict_face(image_path, 
#                  embedding, 
#                  dataset, 
#                  threshold=.1, 
#                  distance_metric='cosine',
#                  output_folder='output',
#                  base_path='/gdrive/My Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'):
#   people = get_names(dataset)
#   y_hat = clean_name(image_path)
#   distances = []
  
#   assert y_hat in people, "The picture is not a part of the test set."
  
#   for person in people:
#     person_path = os.path.join(base_path, person)
#     person_path = os.path.join(person_path, output_folder)
#     faces = os.listdir(person_path)
#     faces = [os.path.join(person_path, face) for face in faces]
    
#     if person == y_hat:
#       faces = list(filter(lambda x: is_diff(x, image_path), faces))
      
#     temp_dist = []
#     for face in faces:
#       face_embedding = np.load(face)
#       d = distance(embedding, face_embedding, distance_metric=distance_metric)
#       temp_dist.append(d)
      
#     distances.append(min(d))
    
#   distances = np.array(distances)
#   min_distance_idx = distances.argmin()
#   y = people[min_distance_idx]
  
#   return y == y_hat, y

In [0]:
os.chdir('/gdrive/My Drive/FaceRecognition/src')
import utils
import tensorflow as tf

In [0]:
sess = tf.Session()
utils.load_model('/gdrive/My Drive/FaceRecognition/models/facenet/20180402-114759/20180402-114759.pb')

# Placeholders
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]

In [0]:
pred_true = 0
name_true = []
length = X_test.shape[0]

for image_path in progressbar.progressbar(X_test):
  image = misc.imread(image_path)
  
  feed_dict = { images_placeholder: image.reshape((1, *image.shape)), 
                phase_train_placeholder: False }
  embeddings_array = np.zeros((1, embedding_size))
  embeddings_array = sess.run(embeddings, feed_dict=feed_dict)
  
  match, prediction, _ = predict_face(image_path, embeddings_array, dataset, threshold=0.1)
  
  if match:
    pred_true += 1
    name_true.append(clean_name(image_path))
  
acc = pred_true / length

print('[ACCURACY] %.2f%%' % (acc*100))

In [0]:
name_true

## Analyse the test set

In [0]:
X_test = np.load('/gdrive/My Drive/FaceRecognition/datasets/lfw/xtest.npy')
y_test = np.load('/gdrive/My Drive/FaceRecognition/datasets/lfw/ytest.npy')

In [0]:
im_path = X_test[628]
image = misc.imread(im_path)
  
feed_dict = { images_placeholder: image.reshape((1, *image.shape)), 
              phase_train_placeholder: False }
embeddings_array = np.zeros((1, embedding_size))
embeddings_array = sess.run(embeddings, feed_dict=feed_dict)

match, prediction, person = predict_face(im_path, embeddings_array, dataset, threshold=0.1)

In [0]:
person

In [0]:
prediction

In [0]:
os.chdir('/gdrive/My Drive/FaceRecognition/models/mobile-net/best_model')

In [14]:
os.listdir()

['mobilenetv1_v12.h5', 'mobilenetv1_v12.npy']

In [0]:
model = models.load_model('mode')